# Interactivity

In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from ipywidgets import widgets, Layout
import folium
from IPython.display import clear_output

In [2]:
# Get cleaned gun violence master dataset 
directory = os.path.dirname(str(Path().resolve()) + '\\')
path = os.path.join(directory, 'data', 'gun-violence-data-cleaned.csv')
gunViolence = pd.read_csv(path)

In [35]:
# Simple Overlays (no Data)

def embed_map(m):
    from IPython.display import IFrame

    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

folium_map = folium.Map(location=[35, -97],
                        zoom_start=4,
                        tiles="Mapbox Bright")
folium.GeoJson('us_counties_20m.json', name='US Counties').add_to(folium_map)
folium.GeoJson('us_states_20m.json', name='US States').add_to(folium_map)
folium.LayerControl().add_to(folium_map)
embed_map(folium_map)

In [36]:
# Widget Demonstration Code

button = widgets.Button(description = "Go!")
slider = widgets.IntSlider(min=0,max=5000,step=100,value=100,description='Number of Incidents:',layout=Layout(width='40%'),style={'description_width': 'initial'})
display(slider)
display(button)

def on_button_clicked(b):
    clear_output()
    display(slider)
    display(button)
    folium_map = folium.Map(location=[35, -97],
                            zoom_start=4,
                            tiles="Mapbox Bright")
    for i, row in gunViolence.head(slider.value).iterrows():
        marker = folium.CircleMarker(location=[row['latitude'], row['longitude']], radius = (row['n_killed'] + row['n_injured']))
        marker.add_to(folium_map)
    display(folium_map)

button.on_click(on_button_clicked)

IntSlider(value=900, description='Number of Incidents:', layout=Layout(width='40%'), max=5000, step=100, style…

Button(description='Go!', style=ButtonStyle())